In [1]:
from azureml.core import Workspace
subscription_id = "15ae9cb6-95c1-483d-a0e3-b1a1a3b06324"
resource_group = "nlpinterpret"
workspace_name = "performanceBenchmarking"
workspace_region = "eastus2"

In [2]:
try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")



Workspace configuration succeeded. Skip the workspace creation steps below


In [3]:
experiment_name = 'performance-benchmarking-remote-vm'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)


import os
script_folder = './vm-run'
os.makedirs(script_folder, exist_ok=True)

In [14]:
from azureml.core.compute import ComputeTarget, RemoteCompute
from azureml.core.compute_target import ComputeTargetException

username = "nlp_text"
address = "40.71.179.131"

compute_target_name = 'benchPerf'
#if you want to connect using SSH key instead of username/password you can provide parameters private_key_file and private_key_passphrase 
try:
    attached_dsvm_compute = RemoteCompute(workspace=ws, name=compute_target_name)
    print('found existing:', attached_dsvm_compute.name)
except ComputeTargetException:
    attach_config = RemoteCompute.attach_configuration(address=address,
                                                       ssh_port=22,
                                                       username=username,
                                                       private_key_file='./.ssh/id_rsa')
    attached_dsvm_compute = ComputeTarget.attach(workspace=ws,
                                                 name=compute_target_name,
                                                 attach_configuration=attach_config)
    attached_dsvm_compute.wait_for_completion(show_output=True)

found existing: benchPerf


In [ ]:
from azureml.core import ScriptRunConfig
from uuid import uuid4

src = ScriptRunConfig(source_directory=script_folder, 
                      script='../scripts/test_run.py', 
                      # pass the dataset as a parameter to the training script
                    #   arguments=['--data-folder',  
                    #              dataset.as_named_input('diabetes').as_mount('/tmp/{}'.format(uuid4()))]
                     ) 

src.run_config.framework = "python"
src.run_config.target = attached_dsvm_compute.name


run = exp.submit(config=src)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

run.wait_for_completion(show_output=True)